In [1]:
sc.install_pypi_package("boto3")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1703945185492_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
from pyspark.sql import SparkSession
import boto3
from pyspark.sql.functions import *
from datetime import datetime, timedelta
from pyspark.sql.types import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession.builder.appName('tripdataanalysis').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
source_bucket, target_bucket = "staging-trip-data", "processed-trip-data"
session = boto3.Session()
s3 = session.client('s3')
dw_bucket_path="s3://"+target_bucket+"/Dimension/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
schema = StructType([
    StructField("Id", IntegerType(), True),
    StructField("SharedRequestFlag", StringType(), True),
    StructField("SharedMatchFlag", StringType(), True),
    StructField("WavRequestFlag", StringType(), True),
    StructField("WavMatchFlag", StringType(), True)
])


data = [(1, 'Y','Y','Y','Y'),
        
        (2, 'Y', 'Y','Y','N'),
        (3,'N','Y','Y','Y'),
        (4,'Y','N','Y','Y'),
        (5,'Y','Y','N','Y'),
        
        (6, 'Y', 'Y','N','N'),
        (7, 'N', 'N','Y','Y'),
        
        (8, 'Y', 'N','Y','N'),
        (9, 'N', 'Y','N','Y'),
        
        (10, 'N', 'Y','Y','N'),
        (11, 'Y', 'N','N','Y'),
        
        (12, 'N', 'Y','Y','Y'),
        (13, 'Y', 'N','Y','Y'),
        (14, 'Y', 'Y','N','Y'),
        (15, 'Y', 'Y','Y','N'),
       
        (16, 'N', 'N','N','N')
       
       ]
file_name='dim_flags'
dim_flags = spark.createDataFrame(data, schema)
dim_flags.repartition(1).write.mode("overwrite").parquet(dw_bucket_path+file_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
schema = StructType([
    StructField("PaymentTypeId", IntegerType(), True),
    StructField("PaymentType", StringType(), True)
])
data=([1,'Credit card'],[2,'Cash'],[3,'No charge'],[4,'Dispute'],[5,'Unknown'],[6,'Voided trip'])
dim_paymenttype=spark.createDataFrame(data, schema)
file_name='dim_paymenttype'
dim_paymenttype.repartition(1).write.mode("overwrite").parquet(dw_bucket_path+file_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
schema = StructType([
    StructField("RateCodeId", IntegerType(), True),
    StructField("RateCode", StringType(), True)
])
data=([1,'Standard rate'],[2,'JFK'],[3,'Newark'],[4,'Nassau or Westchester'],[5,'Negotiated fare'],[6,'Group ride'])
dim_ratecode=spark.createDataFrame(data, schema)
file_name='dim_ratecode'
dim_ratecode.repartition(1).write.mode("overwrite").parquet(dw_bucket_path+file_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
schema = StructType([
    StructField("VendorId", IntegerType(), True),
    StructField("Vendor", StringType(), True)
])
data=([1,'Creative Mobile Technologies'],[2,'VeriFone Inc.'])
dim_vendor=spark.createDataFrame(data, schema)
file_name='dim_vendor'
dim_vendor.repartition(1).write.mode("overwrite").parquet(dw_bucket_path+file_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
dim_zone_source = "s3://"+source_bucket+"/taxi_zone_lookup.csv"
dim_zone=spark.read.csv(dim_zone_source,header=True)
file_name='dim_zone'
dim_zone.repartition(1).write.mode("overwrite").parquet(dw_bucket_path+file_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
start_date = datetime(2009, 1, 1)
end_date = datetime(2025, 12, 31)

# Calculate the number of days between start and end date
num_days = (end_date - start_date).days

# Create a DataFrame with a range of dates
date_range_df = spark.range(0, num_days + 1).selectExpr(f"cast(id as int) as id")

date_range_df = date_range_df.withColumn(
    "date",
    (to_date(lit(start_date)) + col("id")).cast("date")
)

# Extract various date-related columns
dim_date = date_range_df.select(
    col("date").alias("full_date"),
    year("date").alias("year"),
    quarter("date").alias("quarter"),
    month("date").alias("month"),
    dayofmonth("date").alias("day"),
    dayofweek("date").alias("day_of_week"),
    weekofyear("date").alias("week_number")
    # You can add more date-related columns as needed
)

# Add isweekday and isweekend columns
dim_date = dim_date.withColumn(
    "isweekday",
    when((col("day_of_week") >= 2) & (col("day_of_week") <= 6), 1).otherwise(0)
)
dim_date = dim_date.withColumn("isweekend", when(col("isweekday") == 0, 1).otherwise(0))

# dim_date = dim_date.withColumn("DateId", col("index")).drop("index")
# # Show the Date Dimension DataFrame

dim_date=dim_date.withColumn("DateId",concat(col('year'),col('month'),col('day')))

dim_date=dim_date.withColumn("DateId", col("DateId").cast("integer"))

file_name='dim_date'
dim_date.repartition(1).write.mode("overwrite").parquet(dw_bucket_path+file_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
hours_range = range(0, 24)
minutes_range = range(0, 60)

# Create a DataFrame with hours and minutes
dim_time = spark.createDataFrame(
    [(hour, minute) for hour in hours_range for minute in minutes_range],
    ["hour", "minute"]
)

dim_time=dim_time.withColumn("TimeId",concat(col("hour"), col("minute")))
# Show the Time Dimension DataFrame

file_name='dim_time'
dim_time.repartition(1).write.mode("overwrite").parquet(dw_bucket_path+file_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
schema = StructType([
    StructField("LicenseNumber", StringType(), True),
    StructField("LicenseHolder", StringType(), True)
])
data=(['HV0002','Juno'],['HV0003','Uber'],['HV0004','Via'],['HV0005','Lyft'])
dim_license=spark.createDataFrame(data, schema)
file_name='dim_license'
dim_license.repartition(1).write.mode("overwrite").parquet(dw_bucket_path+file_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
dim_flags.repartition(1).write.mode("overwrite").parquet(dw_bucket_path+file_name)